In [3]:
# notebooks/eda.ipynb (or whatever notebook)
from datasets import load_dataset
import os
from pathlib import Path

# Get project root directory (parent of notebooks/)
project_root = Path.cwd().parent
data_dir = project_root / "data"

data_dir.mkdir(parents=True, exist_ok=True)

dataset = load_dataset("deepcopy/MathWriting-human")
dataset.save_to_disk(str(data_dir / "mathwriting"))

Saving the dataset (0/3 shards):   0%|          | 0/229864 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7644 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15674 [00:00<?, ? examples/s]